In [64]:
import os 
os.environ.setdefault('AWS_PROFILE', 'ghactivity')

'ghactivity'

In [65]:
!aws s3 ls s3://sgghactivity


In [66]:
!aws s3 rm  s3://sgghactivity/ --recursive

In [67]:
import boto3

dynamo_db=boto3.resource('dynamodb')
table=dynamo_db.Table('jobs')
table.delete_item(Key={'job_id':'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': 'U2K48F17KVCKR83E3P1EKUJR6JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 25 Jul 2022 16:29:10 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'U2K48F17KVCKR83E3P1EKUJR6JVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [68]:

dynamo_db=boto3.resource('dynamodb')
table=dynamo_db.Table('jobs')
baseline_days=table.get_item(Key={'job_id':'ghactivity_ingest'})['Item']['baseline_days']

In [69]:
type(baseline_days)

decimal.Decimal

In [70]:
int(baseline_days)

6

In [71]:
from datetime import datetime as dt 
from datetime import timedelta as td

In [72]:
next_file=f'{dt.strftime(dt.now().date() - td(days=int(baseline_days)), "%Y-%m-%d")}-0.json.gz'

In [73]:
next_file

'2022-07-19-0.json.gz'

In [74]:
import requests
res=requests.get(f'https://data.gharchive.org/{next_file}')

In [75]:
import boto3
s3_client=boto3.client('s3')

In [38]:
# s3_client.put_object

Signature: s3_client.put_object(*args, **kwargs)
Docstring:
Adds an object to a bucket. You must have WRITE permissions on a bucket to add an object to it.

 

Amazon S3 never adds partial objects; if you receive a success response, Amazon S3 added the entire object to the bucket.

 

Amazon S3 is a distributed system. If it receives multiple write requests for the same object simultaneously, it overwrites all but the last object written. Amazon S3 does not provide object locking; if you need this, make sure to build it into your application layer or use versioning instead.

 

To ensure that data is not corrupted traversing the network, use the ``Content-MD5`` header. When you use this header, Amazon S3 checks the object against the provided MD5 value and, if they do not match, returns an error. Additionally, you can calculate the MD5 while putting an object to Amazon S3 and compare the returned ETag to the calculated MD5 value.

 

.. note::

   

   
  * To successfully complete the

In [76]:
bucket_name='sgghactivity'

In [77]:
upload_res=s3_client.put_object(
    Bucket=bucket_name,
    Key=f'landing/{next_file}',
    Body=res.content
   
    )

In [78]:
upload_res

{'ResponseMetadata': {'RequestId': '37KAW6SSPJDFEHRW',
  'HostId': 'NmM+zF0N6H7ISWBnI2KRFYVvLWB0QxG+HozNXdosdU+HqrCN2w9ReZ87TXt1AOH1LmWuDZHlNhY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NmM+zF0N6H7ISWBnI2KRFYVvLWB0QxG+HozNXdosdU+HqrCN2w9ReZ87TXt1AOH1LmWuDZHlNhY=',
   'x-amz-request-id': '37KAW6SSPJDFEHRW',
   'date': 'Mon, 25 Jul 2022 16:29:30 GMT',
   'etag': '"e2558f5871a9e1cc2feb06cce981419b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e2558f5871a9e1cc2feb06cce981419b"'}

In [79]:
import pandas as pd

In [85]:
df=pd.read_json(
    f's3://sgghactivity/landing/{next_file}',
    lines=True,
    orient='records'

)

FileNotFoundError: sgghactivity/landing/2022-07-19-0.json.gz

In [86]:
df.shape

(131453, 8)